### Documentar un inicio del etl. de donde sacamos los datos?, es big data o data que se puede procesar? donde estan los archivos que aparecen en las rutas? .

In [1]:
# Librerias Necesarias 
import pandas as pd
import os

Los 11 archivos Jsons estaticos de Google, los juntamos en un solo archivo.

In [ ]:
list_sitios_path = [
    'datasets/google/sitios/1.json',
    'datasets/google/sitios/2.json',
    'datasets/google/sitios/3.json',
    'datasets/google/sitios/4.json',
    'datasets/google/sitios/5.json',
    'datasets/google/sitios/6.json',
    'datasets/google/sitios/7.json',
    'datasets/google/sitios/8.json',
    'datasets/google/sitios/9.json',
    'datasets/google/sitios/10.json',
    'datasets/google/sitios/11.json'
]

for i in range(11):
    df = pd.read_json(list_sitios_path[i], lines=True)
    if i == 0:
        sitios = df.copy()
        del df
    else:
        g_sitios = pd.concat([sitios, df], axis=0, ignore_index=True)
        del df

# Eliminamos filas que repitan la clave primaria y unica        
g_sitios = g_sitios.drop_duplicates(subset=['gmap_id'])

Para optimizacion de espacio y procesamiento, estos Jsons los transformamos a formato Parquet

In [ ]:
g_sitios.to_parquet('datasets/google/sitios/sitios_google_columnas_originales.parquet')

proximamente seguimos en grupo, deje un ejemplo de inicio para mejorar la documentacion y estructurar un etl de cada tabla. asignando un numero de orden a cada etl 